## Data quality check / cleaning / preparation 

### Data quality check and cleaning
*By Joseph Prette*

1. The following section briefly explores the basics of the data
2. Removes row of missing values from end of dataframe
3. Explores columns which feature a rare category that may be of high influence

In [ ]:
# Load necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Import file
train = pd.read_csv('./Data/expanded', delimiter='\t', header=None, names=['characteristics'])
train = train['characteristics'].str.split(',', expand=True)
train = train.drop(range(7)).reset_index(drop=True)

# Implement descriptive column names
column_names = ['edibility','cap_shape', 'cap_surface', 'cap_color', 'bruises', 
                'odor', 'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 
                'stalk_shape', 'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 
                'stalk_color_above_ring', 'stalk_color_below_ring', 'veil_type', 'veil_color', 
                'ring_number', 'ring_type', 'spore_print_color', 'population','habitat']
train.columns = column_names
train.head()

In [ ]:
# Get shape of data
original_shape = train.shape
print('The dataset has {} entries, with {} characteristics'.format(original_shape[0], original_shape[1]))

pd.set_option('display.max_columns', None)
train.describe()

In [ ]:
# Find number of missing values in each column
train.isna().sum()

In [ ]:
# Show rows with missing columns
train[train.isnull().any(axis=1)]

In [ ]:
# drop rows (in this case just the one row) containing any NA or None values
train = train.drop(train[train.isnull().any(axis=1)].index)

# show shape has changed:
updated_shape = train.shape
print('The dataset has {} entries, with {} characteristics'.format(updated_shape[0], updated_shape[1]))
print('{} row(s) were removed'.format(original_shape[0] - updated_shape[0]))
print('{} column(s) were removed'.format(original_shape[1] - updated_shape[1]))

In [ ]:
# view the updated description of the data
train.describe()

In [ ]:
# rare is commonly defined as < .1%, using .1% as threshold for values.
col_with_rare = list()
for col_name in train.columns:
    for n in range(len(train[col_name].value_counts())):
        if train[col_name].value_counts()[n] / train.shape[0] < 0.001:
            col_with_rare.append(col_name)
            #print(train[col_name].value_counts())
            #print()
            break;

col_with_rare

### Data Preparation
*By Jackson Bremen*

1. Verified data is not severely imbalanced
2. Implemented dummy variable for edibility
3. Split given data into a training (80%) and test (20%) dataset

In [ ]:
train.edibility.value_counts()

In [ ]:
# get dummy variables for two-level response
def var_transform (data):
    data['edibility']=data['edibility'].apply(lambda x: 1 if x=='EDIBLE' else 0)
    # put further variable transformation here 
    return data

In [ ]:
# Implement transformation and verify change was successful
var_transform(train).head()

In [ ]:
# Creating training and test datasets
np.random.seed(2)
splitted_train = train.sample(round(train.shape[0]*0.8))
test = train.drop(splitted_train.index)
train = splitted_train

In [ ]:
print(train.shape)
print(test.shape)

### Exploratory data analysis
*By Chanel Sun and Jackson Bremen*

1. Discovered `odor` to be very strong predictor for edibility, with certain odors having all or none edible, and just one with ~97% edible
2. Decided to build model using only `odor`, but acknowledge the limitations and will build second without `odor`
3. `veil-type` has only one level, 'PARTIAL', meaning it will have no effect on edibility, so this variable will not be used.
4. We observed that the barplots of `stalk_surface_above_ring` and `stalk_surface_below_ring`, as well as `stalk_color_above_ring` and `stalk_color_below_ring`, showed a high degree of similarity. One from each will be dropped using SelectKBest.
5. Use VIF to determine high influence

In [ ]:
# Create list of the prediction variables; all but the first column (edibility)
predictors = train.columns[1:]

# Display the edibility of each cateogry of each prediction variable
fig, axs = plt.subplots(6, 4, figsize=(20, 20))
for i, predictor in enumerate(predictors):
    row = i // 4
    col = i % 4
    data = train[['edibility', predictor]].groupby([predictor], as_index=False).mean()
    sns.barplot(x=predictor, y='edibility', data=data, ax=axs[row, col])
    axs[row, col].set_title(f'{predictor}')
    axs[row, col].set_xlabel('')
    axs[row, col].set_ylabel('% edibility')
    for rect in axs[row, col].patches:
        height = rect.get_height()
        axs[row, col].annotate(f'{height:.1%}', xy=(rect.get_x() + rect.get_width() / 2, height), 
                               xytext=(0, 3), textcoords='offset points', ha='center', va='bottom')
plt.tight_layout()
plt.show()

In [ ]:
# Make temporary dataset without unnecessary columns, save response variable column as Y
X = train.drop(['edibility', 'veil_type', 'stalk_surface_below_ring', 'stalk_color_below_ring'], axis=1)
Y = train['edibility']

In [ ]:
# Implement dummy variables for all columns since they are all categorical
X = pd.get_dummies(X, columns=['cap_shape', 'cap_surface', 'cap_color', 'bruises', 
                'odor', 'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 
                'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',  
                'stalk_color_above_ring', 'veil_color', 'ring_number', 'ring_type', 
                'spore_print_color', 'population','habitat'], drop_first=True)
X.shape

In [ ]:
# Get variable inflation factors
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["Predictor"] = X.columns
vif_inf = vif[np.isinf(vif['VIF Factor'])]
vif_inf

## Developing the model

### Model Constrution

#### Model 1: `odor`
*By Chanel Sun and Lucy Han*

In [ ]:
#Function to compute confusion matrix and prediction accuracy
def confusion_matrix_data(data,actual_values,model,cutoff=0.5):
    pred_values = model.predict(data)
    bins=np.array([0,cutoff,1])
    cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
    cm_df = pd.DataFrame(cm)
    cm_df.columns = ['Predicted 0','Predicted 1']
    cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
# Calculate the accuracy
    accuracy = (cm[0,0]+cm[1,1])/cm.sum()
    fnr = (cm[1,0])/(cm[1,0]+cm[1,1])
    precision = (cm[1,1])/(cm[0,1]+cm[1,1])
    fpr = (cm[0,1])/(cm[0,0]+cm[0,1])
    tpr = (cm[1,1])/(cm[1,0]+cm[1,1])
    fpr_roc, tpr_roc, auc_thresholds = roc_curve(actual_values, pred_values)
    auc_value = (auc(fpr_roc, tpr_roc))# AUC of ROC
    sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g')
    plt.ylabel("Actual Values")
    plt.xlabel("Predicted Values")
    print("Classification accuracy = {:.1%}".format(accuracy))
    print("Precision = {:.1%}".format(precision))
    print("TPR or Recall = {:.1%}".format(tpr))
    print("FNR = {:.1%}".format(fnr))
    print("FPR = {:.1%}".format(fpr))
    print("ROC-AUC = {:.1%}".format(auc_value))

In [ ]:
train_1 = train.copy()

# If odor is equal to ALMOND, ANISE, or NONE, set it to 0, else 1
train_1['odor'] = train_1['odor'].apply(lambda x: 0 if x in ['ALMOND', 'ANISE', 'NONE'] else 1).astype(int)

# Train a linear regression model
model_1 = smf.logit(formula = 'edibility ~ odor', data=train_1).fit()

confusion_matrix_data(test_1[['odor']], test_1['edibility'], model_1)

#### Model 2: Complex Model
*By Chanel Sun and Lucy Han*

In [ ]:
train_2 = train.copy()

train_2['edibility'] = train_2['edibility'].astype(int)

columns_to_use = ['cap_shape', 'cap_surface', 'cap_color', 'bruises',
                  'gill_attachment', 'gill_spacing', 'gill_size', 'stalk_color_above_ring']

formula = 'edibility ~ ' + ' + '.join(columns_to_use)

# Train a linear regression model
model_2 = smf.logit(formula = formula, data = train_2).fit()

# Test the model on the test set
test_2 = test.copy()
test_2['edibility'] = test_2['edibility'].astype(int)

confusion_matrix_data(test_2[columns_to_use], test_2['edibility'], model_2, cutoff=0.4)

In [ ]:
def plot_precision_vs_threshold(precision, recall, thresholds):
    plt.plot(thresholds, precision[:-1], "b--", label="Precision")
    plt.plot(thresholds, recall[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylabel("Precision/Recall")

In [ ]:
# Find the optimal cutoff
y = train_2['edibility']
y_predict = model_2.predict(train_2)
p, r, prc_thresholds = precision_recall_curve(train_2['edibility'], model_2.predict(train_2))

fpr, tpr, thresholds = roc_curve(y, y_predict)

plot_precision_vs_threshold(p, r, prc_thresholds)

maximizing_threshold = thresholds[np.argmax(p[:-1] * r[:-1])]

print(f"When the threshold is {round(maximizing_threshold, 5)}")
print(f"the precision is {round(p[np.argmin(np.abs(thresholds - maximizing_threshold))], 5)}")
print(f"the recall is {round(r[np.argmin(np.abs(thresholds - maximizing_threshold))], 5)}")
print(f"maximizing_threshold is {round(maximizing_threshold, 5)}")

# Find the point in precision where threshold is closest to maximizing_threshold

y0 = p[np.argmin(np.abs(thresholds - maximizing_threshold))]
y1 = r[np.argmin(np.abs(thresholds - maximizing_threshold))]

sns.scatterplot(x=[maximizing_threshold], y=[y0], color='red', s=100)
sns.scatterplot(x=[maximizing_threshold], y=[y1], color='red', s=100)

### Code fitting the final model

#### Model 1: `odor`

In [ ]:
model_1 = smf.logit(formula = 'edibility ~ odor', data=train_1).fit()

#### Model 2: Complex Model 

In [ ]:
columns_to_use = ['cap_shape', 'cap_surface', 'cap_color', 'bruises',
                  'gill_attachment', 'gill_spacing', 'gill_size', 'stalk_color_above_ring']

formula = 'edibility ~ cap_shape + cap_surface + cap_color + bruises + gill_attachment + gill_spacing + gill_size + stalk_color_above_ring'

model_2 = smf.logit(formula = formula, data = train_2).fit()